# Named Entity Recognition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennethEnevoldsen/DaCy/blob/master/docs/performance.ner.ipynb)


This page examines the performance of competing models for Danish named entity recognition over multiple datasets. Performance is not limited to 
accuracy, but also includes domain generalization, biases and robustness. This page is also a notebook, which open and replicate the results.

## State-of-the-Art comparison
To our knowledge there exists three datasets for Danish named entity recognition;

1) DaNE {cite}`hvingelby2020dane`, which uses the simple annotation scheme of CoNLL 2003 {cite}`missing` with the entities; *person*, *location*, *organization*, and *miscellaneus*.
2) DANSK {cite}`missing`, which uses the extensive annotation scheme similar to that of OntoNotes 5.0 {cite}`missing` including more that 16 entity types.
3) and DAN+ {cite}`missing`, which also uses the annotation scheme of CoNLL 2003, but allows for nested entities for instance *Aarhus Universitet*, where *Aarhus* is a location and *Aarhus Universitet* is an organization.

In this comparison we will be examing performance on DaNE and DANSK, but as no known models have been trained on Danish nested entities, we will not be comparing performance on DAN+.


```{admonition} Measuring Performance
Typically when measuring performance on these benchmark it is normal to feed the model the gold standard tokens. While this allows for easier comparisons of modules and architectures, it inflates the performance metrics. Further, it does not proberly reflect what you are really interested in:
*the performance you can expect when you apply the model to data of a similar type*. Therefore we estimate the model is given no prior knowledge of the data, and only the raw text is fed to the model. Thus the performance metrics might be slightly different compared to e.g. DaNLP.
```

### DaNE: Simple Named Entity Recognition
As already stated DaNE uses an extraction from the CoNLL 2003 dataset, which is as follows {cite}`hvingelby2020dane`:


| Entity | Description |
|--------------|-------------|
| LOC          | includes locations like cities, roads and mountains, as well as both public and commercial places like specific buildings or meeting points, but also abstract places. |
| PERSON | consists of names of people, fictional characters, and animals. The names includes aliases. |
| ORG | can be summarized as all sorts of organizations and collections of people, ranging from companies, brands, political movements, governmental bodies and clubs. |
| MISC | is a broad category of e.g. events, languages, titles and religions, but this tag also includes words derived from one of the four tags as well as words for which one part is from one of the three other tags. |

Here is an example from the dataset:

In [ ]:
import spacy 
from spacy.tokens import Span
from spacy import displacy
text = """To kendte russiske historikere Andronik Mirganjan og Igor Klamkin tror ikke, at Rusland kan udvikles uden en "jernnæve"."""
nlp = spacy.blank("da")
doc = nlp(text)
doc.ents = [ # type: ignore
    Span(doc, 2, 3, label="MISC"),
    Span(doc, 4, 6, label="PERSON"),
    Span(doc, 7, 9, label="PERSON"),
    Span(doc, 13, 14, label="LOC"),
]

displacy.render(doc, style="ent")

The table below shows the performance of Danish language processing pipelines scored on the DaNE test set. The best scores in each category are highlighted with bold and the second best is underlined.

In [ ]:
from pathlib import Path
import pandas as pd
from dacy.datasets import dane
from performance_testing_utils.ner_sota import apply_models, MDL_GETTER_DICT, apply_models, create_table



save_folder = Path("performance_tables/ner")
save_folder.mkdir(exist_ok=True, parents=True)
dataset = dane(splits = "test")

tables = []
for model_name, getter in MDL_GETTER_DICT.items():
    print(model_name)
    save_folder = save_folder / f"{model_name}_dane.csv"
    if not save_folder.exists():
        nlp = getter()
        result_df = apply_models([(model_name, nlp)], dataset, augmenters, n_rep=20)  # type: ignore
        result_df.to_csv(save_folder, index=False)
    else:
        print("- Already exists, loading in dataframe")
        result_df = pd.read_csv(save_folder)
    tables.append(result_df)

df = pd.concat(tables)

s = create_table(df)

s

```{admonition} Model X is missing?
:note:

These tables are continually updated and thus we try to limit the number of models to only the most relevant Danish models. Therefore models like Polyglot with strict requirements and consistently worse performance are excluded. If you want to see a specific model, please open an issue on GitHub.
```




## DANSK: Fine-grained Named Entity Recognition

DANSK is annotated from the Danish Gigaword Corpus {cite}`missing` and a wide variety of domains including conversational, legal, news, social media, web content,  wiki's and Books. Dansk follows includes the following labels:


|  Entity        |             Description                                         |
| -------- | ---------------------------------------------------- |
| PERSON   | People, including fictional                          |
| NORP     | Nationalities or religious or political groups       |
| FACILITY | Building, airports, highways, bridges, etc.          |
| ORGANIZATION | Companies, agencies, institutions, etc.              |
| GPE      | Countries, cities, states.                           |
| LOCATION | Non-GPE locations, mountain ranges, bodies of water  |
| PRODUCT  | Vehicles, weapons, foods, etc. (not services)        |
| EVENT    | Named hurricanes, battles, wars, sports events, etc. |
| WORK OF ART | Titles of books, songs, etc.                         |
| LAW      | Named documents made into laws                       |
| LANGUAGE | Any named language                                   |

As well as annotation for the following concepts:

|   Entity       |   Description                                         |
| -------- | ------------------------------------------- |
| DATE     | Absolute or relative dates or periods       |
| TIME     | Times smaller than a day                    |
| PERCENT  | Percentage (including "*"%)                |
| MONEY    | Monetary values, including unit             |
| QUANTITY | Measurements, as of weight or distance      |
| ORDINAL  | "first", "second"                           |
| CARDINAL | Numerals that do no fall under another type |




In [1]:
# from pathlib import Path
# import pandas as pd
# from performance_testing_utils.ner_sota import apply_models, MDL_GETTER_DICT, apply_models, create_table, load_dansk


# save_folder = Path("performance_tables/ner")
# save_folder.mkdir(exist_ok=True, parents=True)
# dansk = load_dansk()

# tables = []
# for model_name, getter in MDL_GETTER_DICT.items():
#     print(model_name)
#     save_folder = save_folder / f"{model_name}_dansk.csv"
#     if not save_folder.exists():
#         nlp = getter()
#         result_df = apply_models([(model_name, nlp)], dataset, augmenters, n_rep=20)  # type: ignore
#         result_df.to_csv(save_folder, index=False)
#     else:
#         print("- Already exists, loading in dataframe")
#         result_df = pd.read_csv(save_folder)
#     tables.append(result_df)

# df = pd.concat(tables)

# s = create_table(df)

# s

ModuleNotFoundError: No module named 'performance_testing_utils.ner_sota'

## Biases

To examine the biases in Danish models we use augmentation to replace names in the Danish dataset DaNE {cite}`hvingelby2020dane`, this approach
is similar to that introduced in the initial DaCy paper {cite}`enevoldsen2021dacy`.

Here is a short example of how the augmentation might look like:


````{admonition} Example

```{admonition} Original
:class: note


Peter Schmeichel mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```

```{admonition} Female name augmentation
:class: important

Anne Østergaard mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```
````


In [8]:
from pathlib import Path
import pandas as pd
from performance_testing_utils.ner_bias_utils import apply_models, MDL_GETTER_DICT, apply_models, create_table, get_augmenters
from dacy.datasets import dane

augmenters = get_augmenters()
save_folder = Path("performance_tables/ner")
save_folder.mkdir(exist_ok=True, parents=True)
dataset = dane(splits = "test")

tables = []
for model_name, getter in MDL_GETTER_DICT.items():
    print(model_name)
    model_name_ = model_name.replace("/", "_")
    save_path = save_folder / f"{model_name_}_bias.csv"
    if not save_path.exists():
        nlp = getter()
        result_df = apply_models([(model_name, nlp)], dataset, augmenters, n_rep=20)  # type: ignore
        result_df.to_csv(save_folder, index=False)
    else:
        print("- Already exists, loading in dataframe")
        result_df = pd.read_csv(save_path)
    tables.append(result_df)

df = pd.concat(tables)

s = create_table(df)
s

/Users/au561649/Github/DaCy/src/dacy/datasets/names.py:53: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  names = names.groupby(level=0).apply(lambda x: x / float(x.sum()))
/Users/au561649/Github/DaCy/src/dacy/datasets/names.py:53: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  names = names.group

da_core_news_sm-3.5.0
- Already exists, loading in dataframe
da_core_news_md-3.5.0
- Already exists, loading in dataframe
da_core_news_lg-3.5.0
- Already exists, loading in dataframe
da_core_news_trf-3.5.0
- Already exists, loading in dataframe
da_dacy_small_trf-0.2.0
- Already exists, loading in dataframe
da_dacy_medium_trf-0.2.0
- Already exists, loading in dataframe
da_dacy_large_trf-0.2.0
- Already exists, loading in dataframe
da_dacy_small_ner_fine_grained-0.1.0
- Already exists, loading in dataframe
da_da_dacy_medium_ner_fine_grained-0.1.0
- Already exists, loading in dataframe
da_dacy_large_ner_fine_grained-0.1.0
- Already exists, loading in dataframe
saattrupdan/nbailab-base-ner-scandi
- Already exists, loading in dataframe
alexandrainst/da-ner-base


## Generalization
To examine model generalization, we utilize the [DANSK](https://huggingface.co/datasets/chcaa/DANSK) dataset. This dataset is annotated across many different domains including fiction, web content, social media, wikis, news, legal and conversational data. The original dataset includes annotations corresponding to the ontonotes standard (see [getting started](https://centre-for-humanities-computing.github.io/DaCy/tutorials/basic.html#fine-grained-ner) for the full list). To test the generalization we here convert the annotations to the CoNLL-2003 format using the labels `Person`, `Location`, `Organization`. As CoNLL-2003, `Location` includes cities, roads, mountains, abstract places, specific buildings, and meeting points. Thus the `GPE` (geo-political entity) were converted to `Location`. The `MISC` category in CoNLL-2003 is a diverse category meant to denote all names not in other categories (encapsulating both e.g. events and adjectives such as ”2004 World Cup” and ”Italian”), and is therefore not included.

In [2]:
from performance_testing_utils.generalization_utils import dansk, convert_to_conll_2003, MDL_GETTER_DICT, evaluate_generalization, create_generation_viz

train, dev, test = dansk()
convert_to_conll_2003(train)
convert_to_conll_2003(dev)
convert_to_conll_2003(test)

dataset = train + dev + test

assert set([e.label_ for doc in dataset for e in doc.ents]) == set(["PER", "LOC", "ORG"])

save_folder = Path("performance_tables/ner")
save_folder.mkdir(exist_ok=True, parents=True)

tables = []
# create domains datasets
domains = {}
for doc in dataset:
    domain = doc._.meta["dagw_domain"]
    if domain not in domains:
        domains[domain] = []
    domains[domain].append(doc)

for mdl, getter in MDL_GETTER_DICT.items():
    save_path = save_folder / f"{mdl}_generalization.csv"
    if not save_path.exists():
        nlp = getter()
        result_df = evaluate_generalization(mdl_name =mdl, mdl=nlp, domains_dataset_dict=domains, n_rep=100)
        result_df.to_csv(save_path, index=False)
    else:
        print(f"- {mdl} already exists, loading in dataframe")
        result_df = pd.read_csv(save_path)
    tables.append(result_df)

df = pd.concat(tables)

chart = create_generation_viz(df)
chart

ModuleNotFoundError: No module named 'ner_bias_utils'

In [ ]:
sp_sm = spacy.load("da_core_news_sm")
sp_lg = spacy.load("da_core_news_lg")

mdls = [
    ("spaCy (da_core_news_sm)", sp_sm),
    ("spaCy (da_core_news_lg)", sp_lg),
]
from spacy.training import Example
from spacy.scorer import Scorer
import random
import numpy as np

scorer = Scorer()

def no_misc_getter(doc, attr):
    for ent in doc.ents:
        if ent.label_ != "MISC":
            yield ent

def bootstrap(examples, n_rep=100):
    scores = []
    for i in range(n_rep):
        sample = random.choices(examples, k=len(examples))
        score = scorer.score_spans(sample, getter=no_misc_getter, attr="ents")
        scores.append(score)
    return scores

def compute_mean_and_ci(scores):

    ent_f = [score["ents_f"] for score in scores]
    per_f = [score["ents_per_type"].get("PER", {"f": None})["f"] for score in scores]
    loc_f = [score["ents_per_type"].get("LOC", {"f": None})["f"] for score in scores]
    org_f = [score["ents_per_type"].get("ORG", {"f": None})["f"] for score in scores]

    nam = ["Average F1", "Person F1", "Location F1", "Organization F1"]

    d = {}
    for n, f in zip(nam, [ent_f, per_f, loc_f, org_f]):
        f = [x for x in f if x is not None]
        if len(f) == 0:
            d[n] = {
                "mean": None,
                "ci": None
            }
            continue
        d[n] = {
            "mean": np.mean(f),
            "ci": np.percentile(f, [2.5, 97.5])
        }
    return d


all_examples = {}
rows= []
for mdl_name, mdl in mdls:
    all_examples[mdl_name] = []
    for domain in domains:
        docs = domains[domain]
        model_pred = mdl.pipe([doc.text for doc in docs])
        examples = [Example(predicted=x, reference=y) for x, y in zip(model_pred, docs)]
        all_examples[mdl_name].extend(examples)

        bs_score = bootstrap(examples)
        score = compute_mean_and_ci(bs_score)


        row = {
            "Model": mdl_name,
            "Domain": domain,
            "Average F1": score["Average F1"]["mean"],
            "Person F1": score["Person F1"]["mean"],
            "Location F1": score["Location F1"]["mean"],
            "Organization F1": score["Organization F1"]["mean"],
            "Average F1 CI": score["Average F1"]["ci"],

            "Number of docs": len(docs),
            
        }
        rows.append(row)

# across domains
for mdl in all_examples:
    examples = all_examples[mdl]
    bs_score = bootstrap(examples)
    score = compute_mean_and_ci(bs_score)

    row = {
        "Model": mdl,
        "Domain": "All",
        "Average F1": score["Average F1"]["mean"],
        "Person F1": score["Person F1"]["mean"],
        "Location F1": score["Location F1"]["mean"],
        "Organization F1": score["Organization F1"]["mean"],
        "Average F1 CI": score["Average F1"]["ci"],
        "Number of docs": len(examples),
    }
    rows.append(row)

# write to file
import pandas as pd
df = pd.DataFrame(rows)
df.to_csv("ner_performance.csv", index=False)

In [ ]:
import pandas as pd
import altair as alt

df = pd.DataFrame(rows)

# filter out domains
df = df[df["Domain"] != "danavis"]
df = df[df["Domain"] != "dannet"]
df = df[df["Domain"].notnull()]

df['Average F1 CI Lower'] = df['Average F1 CI'].apply(lambda x: x[0])
df['Average F1 CI Upper'] = df['Average F1 CI'].apply(lambda x: x[1])
df['Average F1 CI Lower'] = pd.to_numeric(df['Average F1 CI Lower'])
df['Average F1 CI Upper'] = pd.to_numeric(df['Average F1 CI Upper'])



selection = alt.selection_point(fields=['Domain'], bind='legend', value=[{'Domain': 'All'}]) # does not work

bind_checkbox = alt.binding_checkbox(name='Scale point size by number of documents: ')
param_checkbox = alt.param(bind=bind_checkbox)

base = alt.Chart(df).mark_point(filled=True).encode(
    # x='Average F1',
    x=alt.X('Average F1', title="F1"),
    y='Model',
    color='Domain',
    size=alt.condition(
        param_checkbox,
        'Number of docs',
        alt.value(100)
    ),
    tooltip=["Model", "Domain", "Average F1", "Person F1", "Location F1", "Organization F1"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.0))
)
error_bars = alt.Chart(df).mark_errorbar(ticks=False).encode(
    # x='Average F1 CI Lower',
    x = alt.X('Average F1 CI Lower', title="F1"),
    x2='Average F1 CI Upper',
    y='Model',
    color='Domain',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.0))
)

chart = error_bars + base

chart.add_params(selection, param_checkbox).properties(
    width=800,
    height=400
)

```{note}
The F1 in the figure denotes the mean bootstrapped F1 score with a 95% confidence interval. The F1 score is calculated on all of the DANSK dataset.
```

## Robustness

In the paper [DaCy: A Unified Framework for Danish NLP](https://github.com/centre-for-humanities-computing/DaCy/blob/main/papers/DaCy-A-Unified-Framework-for-Danish-NLP/readme.md) we conducted a series on augmentation on the DaNE test set to estimate the robustness and biases of DaCy and other Danish language processing pipelines. This page represents only parts of the paper. We recommend reading the paper for a more thorough and nuanced overview.

The augmentation we will be using in this test are performed on the DaNE test set and include the following:

- **Spelling Error**: Intended to similar domains with inconsistent spelling, OCR errors, conversational data, etc.. The augmentation includes a series of smaller augmentation:
  - Keystroke error: The augmentation is used to introduce errors by replacing a character with a character that is close on the keyboard.
  - Character swap: The augmentation is used to introduce errors by swapping two neighboring characters.
  - Token swap: The augmentation is used to introduce errors by swapping two neighboring tokens.
- **Synonym Augmentation**: This augmentation is used to simulate the variation and slight grammatical errors in the language and uses two different methods:
  - Wordnet Synonym replacement: The augmentation replaces a token with a synonym in WordNet while respecting its syntactic role.
  - Embedding Synonym replacement: This augmentation replaces a token with a synonym which tends to appear in similar contexts.
- **Inconsistent Spacing**: This augmentation is used to simulate inconsistent spacing in the language and uses two different methods by either randomly adding or removing spaces.
- **Inconsistent Casing**: This augmentation is used to simulate inconsistent casing in the language and uses two different methods by either randomly capitalizing or lowercasing tokens.
- **Historical Spelling**: This augmentation is used to simulate historical spelling in Danish including ASCII spellings of the letters Æ (Ae), Ø (Oe), and Å (Aa) as well as uppercasing nouns.

For all of the augmentations the probability of an augmentation is set to 2% and the number of augmentations is set to 1. The augmentations are performed using the [augmenty](https://kennethenevoldsen.github.io/augmenty/index.html).

        <!-- "Spelling Error": sim_spelling_error_aug,
        "Inconsistent Casing": inconsistent_casing_aug,
        "Synonym replacement": synonym_aug,
        "Inconsistent Spacing": spacing_aug,
        "Historical Spelling": hist_spelling_aug, -->

Let's start by examining a couple of the augmentations, namely changing out names or introducing plausible keystroke errors.



````{admonition} Example

```{note} Original

Peter Schmeichel mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```

```{important} **2% simulated spelling errors**

ADD ME
```
````

The underlying assumption of making these augmentations is that the tags of the tokens do not change with augmentation. In our case, this includes that "Anna Østergaard" is still a person and that "vonde" can still be considered a verb based on its context.

Based on this, we can assume that if a model performs worse on a certain set of names or with minor spelling variations or errors, we can conclude that the model is vulnerable to such input. For instance, if the model has a hard time when replacing æ, ø, and å with ae, oe, and aa, it might not be ideal to apply to historic texts.

As seen in the example above, while text with 5% keystroke is still readable. However, 15% keystroke errors tests the limit of what humans and models can reasonably be expected to comprehend.

```{important}
**15% keytype errors**

Peter Schmeichel mejer ogsp, at ddt danske landshoof anbo 202q tilhårer gerfenatop0en of lan vinde sen kpmkendw lamp mod England.
```




The following tables show a detailed breakdown of performance for named entity recognition, part-of-speech tagging, and dependency parsing. These show some general trends, some of which include:

- Spelling variations and abbreviated first names consistently reduce performance of all models on all tasks.
- Even simple replacements of æ, ø, and å with ae, oe, and aa lead to notable performance degradation.
- In general, larger models handle augmentations better than small models with DaCy large performing the best.
- The BiLSTM-based models (Stanza and Flair) perform competitively under augmentations and are only consistently outperformed by DaCy large.


# References

```{bibliography}
```